In [17]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import agentops
# from google.colab import userdata
from pydantic import BaseModel, Field
from typing import List
from tavily import TavilyClient
from scrapegraph_py import Client
import os
import json
from dotenv import load_dotenv
load_dotenv()


True

In [20]:


agent = Agent(
    role='Local AI Expert',
    goal='Process information using a local model',
    backstory="An AI assistant running on local hardware.",
    llm=LLM(model="ollama/llama3.2", base_url="http://localhost:127.0.0.1:11500")
)

In [22]:

AgentOps_Key=os.getenv('AgentOPs_Key' )
tavily_key=os.getenv('tavily' )
scrapegraph_Key=os.getenv('scrapegraph' )

In [24]:

agentops.init(
    api_key=AgentOps_Key,
    skip_auto_end_session=True
)

🖇 AgentOps: AgentOps has already been initialized. If you are trying to start a session, call agentops.start_session() instead.


In [55]:
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)
basic_llm = LLM(model="ollama/llama3", base_url="http://127.0.0.1:11434")
search_client = TavilyClient(api_key=tavily_key)
scrape_client = Client(api_key=scrapegraph_Key)

In [56]:
no_keywords = 10

about_company = "TechX is a company that provides AI solutions to help websites refine their search and recommendation systems."

company_context = StringKnowledgeSource(
    content=about_company
)

## Setup Agents

### Agent: A

In [57]:
class SuggestedSearchQueries(BaseModel):
    queries: List[str] = Field(..., title="Suggested search queries to be passed to the search engine",
                               min_items=1, max_items=no_keywords)

search_queries_recommendation_agent = Agent(
    role="Search Queries Recommendation Agent",
    goal="\n".join([
                "To provide a list of suggested search queries to be passed to the search engine.",
                "The queries must be varied and looking for specific items."
            ]),
    backstory="The agent is designed to help in looking for products by providing a list of suggested search queries to be passed to the search engine based on the context provided.",
    llm=basic_llm,
    verbose=True,
    tools=[]
)

search_queries_recommendation_task = Task(
    description="\n".join([
        "TechX is looking to buy {product_name} at the best prices (value for a price strategy)",
        "The campany target any of these websites to buy from: {websites_list}",
        "The company wants to reach all available proucts on the internet to be compared later in another stage.",
        "The stores must sell the product in {country_name}",
        "Generate at maximum {no_keywords} queries.",
        "The search keywords must be in {language} language.",
        "Search keywords must contains specific brands, types or technologies. Avoid general keywords.",
        "The search query must reach an ecommerce webpage for product, and not a blog or listing page."
    ]),
  expected_output = """A JSON object containing a list of suggested search queries. e.g. ["Best Coffee machines deals"]"""
,


    output_json=SuggestedSearchQueries,
    output_file=os.path.join(output_dir, "step_1_suggested_search_queries.json"),
    agent=search_queries_recommendation_agent
)

🖇 AgentOps: Multiple sessions detected. You must use session.create_agent(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management


### Agent: B

In [58]:
class SignleSearchResult(BaseModel):
    title: str
    url: str = Field(..., title="the page url")
    content: str
    score: float
    search_query: str

class AllSearchResults(BaseModel):
    results: List[SignleSearchResult]

@tool
def search_engine_tool(query: str):
    """Useful for search-based queries. Use this to find current information about any query related pages using a search engine"""
    return search_client.search(query)

search_engine_agent = Agent(
    role="Search Engine Agent",
    goal="To search for products based on the suggested search query",
    backstory="The agent is designed to help in looking for products by searching for products based on the suggested search queries.",
    llm=basic_llm,
    verbose=True,
    tools=[search_engine_tool]
)

search_engine_task = Task(
    description="\n".join([
        "The task is to search for products based on the suggested search queries.",
        "You have to collect results from multiple search queries.",
        "Ignore any suspicious links or non-ecommerce single product website links.",
        "Ignore any search results with a confidence score less than {score_th}.",
        "The search results will be used to compare prices of products from different websites.",
    ]),
    expected_output="""JSON object containing the search results.Return the response as a JSON object without including "json" at the beginning.""",
    output_json=AllSearchResults,
    output_file=os.path.join(output_dir, "step_2_search_results.json"),
    agent=search_engine_agent
)

🖇 AgentOps: Multiple sessions detected. You must use session.create_agent(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management


### Agent: C

In [59]:
class ProductSpec(BaseModel):
    specification_name: str
    specification_value: str

class SingleExtractedProduct(BaseModel):
    page_url: str = Field(..., title="The original url of the product page")
    product_title: str = Field(..., title="The title of the product")
    product_image_url: str = Field(..., title="The url of the product image")
    product_url: str = Field(..., title="The url of the product")
    product_current_price: float = Field(..., title="The current price of the product")
    product_original_price: float = Field(title="The original price of the product before discount. Set to None if no discount", default=None)
    product_discount_percentage: float = Field(title="The discount percentage of the product. Set to None if no discount", default=None)

    product_specs: List[ProductSpec] = Field(..., title="The specifications of the product. Focus on the most important specs to compare.", min_items=1, max_items=5)

    agent_recommendation_rank: int = Field(..., title="The rank of the product to be considered in the final procurement report. (out of 5, Higher is Better) in the recommendation list ordering from the best to the worst")
    agent_recommendation_notes: List[str]  = Field(..., title="A set of notes why would you recommend or not recommend this product to the company, compared to other products.")


class AllExtractedProducts(BaseModel):
    products: List[SingleExtractedProduct]


@tool
def web_scraping_tool(page_url: str):
    """
    An AI Tool to help an agent to scrape a web page

    Example:
    web_scraping_tool(
        page_url="https://www.noon.com/egypt-en/15-bar-fully-automatic-espresso-machine-1-8-l-1500"
    )
    """
    details = scrape_client.smartscraper(
        website_url=page_url,
        user_prompt="Extract " + SingleExtractedProduct.model_json_schema() + " From the web page"
    )

    return {
        "page_url": page_url,
        "details": details
    }

scraping_agent = Agent(
    role="Web scraping agent",
    goal="To extract details from any website",
    backstory="The agent is designed to help in looking for required values from any website url. These details will be used to decide which best product to buy.",
    llm=basic_llm,
    tools=[web_scraping_tool],
    verbose=True,
)

scraping_task = Task(
    description="\n".join([
        "The task is to extract product details from any ecommerce store page url.",
        "The task has to collect results from multiple pages urls.",
        "Collect the best {top_recommendations_no} products from the search results.",
    ]),
    expected_output="A JSON object containing products details",
    output_json=AllExtractedProducts,
    output_file=os.path.join(output_dir, "step_3_search_results.json"),
    agent=scraping_agent
)

🖇 AgentOps: Multiple sessions detected. You must use session.create_agent(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management


### Agent: D

In [60]:
procurement_report_author_agent = Agent(
    role="Procurement Report Author Agent",
    goal="To generate a professional HTML page for the procurement report",
    backstory="The agent is designed to assist in generating a professional HTML page for the procurement report after looking into a list of products.",
    llm=basic_llm,
    verbose=True,
)

procurement_report_author_task = Task(
    description="\n".join([
        "The task is to generate a professional HTML page for the procurement report.",
        "You have to use Bootstrap CSS framework for a better UI.",
        "Use the provided context about the company to make a specialized report.",
        "The report will include the search results and prices of products from different websites.",
        "The report should be structured with the following sections:",
        "1. Executive Summary: A brief overview of the procurement process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts.",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for procurement based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        "8. Appendices: Any additional information, such as raw data or supplementary materials.",
    ]),

    expected_output="A professional HTML page for the procurement report.",
    output_file=os.path.join(output_dir, "step_4_procurement_report.html"),
    agent=procurement_report_author_agent,
)

🖇 AgentOps: Multiple sessions detected. You must use session.create_agent(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management


## Run the AI Crew

In [61]:
TechX_crew = Crew(
    agents=[
        search_queries_recommendation_agent,
        search_engine_agent,
        scraping_agent,
        procurement_report_author_agent,
    ],
    tasks=[
        search_queries_recommendation_task,
        search_engine_task,
        scraping_task,
        procurement_report_author_task,
    ],
    process=Process.sequential,
    knowledge_sources=[company_context]
)

Overriding of current TracerProvider is not allowed


In [62]:
crew_results = TechX_crew.kickoff(
    inputs={
        "product_name": "coffee machine for the office",
        "websites_list": ["www.amazon.eg", "www.jumia.com.eg", "www.noon.com/egypt-en"],
        "country_name": "Egypt",
        "no_keywords": 5,
        "language": "English",
        "score_th": 0.10,
        "top_recommendations_no": 10
    }
)

# Agent: Search Queries Recommendation Agent
## Task: TechX is looking to buy coffee machine for the office at the best prices (value for a price strategy)
The campany target any of these websites to buy from: ['www.amazon.eg', 'www.jumia.com.eg', 'www.noon.com/egypt-en']
The company wants to reach all available proucts on the internet to be compared later in another stage.
The stores must sell the product in Egypt
Generate at maximum 5 queries.
The search keywords must be in English language.
Search keywords must contains specific brands, types or technologies. Avoid general keywords.
The search query must reach an ecommerce webpage for product, and not a blog or listing page.


🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().




# Agent: Search Queries Recommendation Agent
## Final Answer: 
{
  "queries": [
    "Best Saeco coffee machines on Amazon Egypt",
    "Jumia Egypt Nespresso coffee machines prices",
    "Tassimo coffee machine deals Noon Egypt",
    "De'Longhi coffee machines on Jumia Egypt",
    "Saeco GranBaristo espresso machine prices on Noon Egypt"
  ]
}


# Agent: Search Engine Agent
## Task: The task is to search for products based on the suggested search queries.
You have to collect results from multiple search queries.
Ignore any suspicious links or non-ecommerce single product website links.
Ignore any search results with a confidence score less than 0.1.
The search results will be used to compare prices of products from different websites.


🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Could not record event - multiple sessions detected. You must use session.record() instead of agentops.record() More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event - multiple sessions detected. You must use session.record() instead of agentops.record() More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management


 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for Search_Engine_Tool
query
  Input should be a valid string [type=string_type, input_value={'description': 'Best Sae...n Egypt', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool search_engine_tool accepts these inputs: Tool Name: search_engine_tool
Tool Arguments: {'query': {'description': None, 'type': 'str'}}
Tool Description: Useful for search-based queries. Use this to find current information about any query related pages using a search engine



# Agent: Search Engine Agent
## Thought: Thought: I need to start by defining my search query and running it through the search engine tool. Then, I'll analyze the results to identify the relevant products and their prices.
## Using tool: search_engine_tool
## Tool Input: 
"{\"query\": {\"description\": \"Best Saeco coffee machines on Amazon Eg

🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Could not record event - multiple sessions detected. You must use session.record() instead of agentops.record() More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management




# Agent: Search Engine Agent
## Thought: I apologize for the mistake earlier. Let me restart the process with the correct approach.
Thought:
## Using tool: search_engine_tool
## Tool Input: 
"{\"query\": \"Best Saeco coffee machines on Amazon Egypt\"}"
## Tool Output: 
{'query': 'Best Saeco coffee machines on Amazon Egypt', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.amazon.eg/-/en/Saeco-Aulika-Office-Freestanding-Grinder/dp/B07BVJ1HLF', 'title': 'Saeco Aulika Office Freestanding Coffee Maker, 4 L, Built-In Grinder ...', 'content': "Search Amazon.eg  Saeco Aulika Office Freestanding Coffee Maker, 4 L, Built-In Grinder, 1300 W, Black: Buy Online at Best Price in Egypt - Souq is now Amazon.eg We don't know when or if this item will be back in stock.Delivering to New Cairo City - Update location Unable to add item to List. This item from this seller is eligible for card payment only on checkout. To calculate the overall star rating and per

🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().




# Agent: Search Engine Agent
## Final Answer: 
Based on the provided Amazon product listings and customer reviews, it appears that some of the top-rated coffee makers and espresso machines in Egypt are from brands like SOKANY, Black & Decker, De'Longhi, and DSP. These products seem to offer features such as portability, waterproof designs, stainless steel construction, adjustable settings, and automatic decalcification.

Some popular options include:

* SOKANY SK-214 500ml Stainless Steel Coffee Machine
* Black & Decker 450w 2 cups coffee maker machine (DCM48)
* De'Longhi ECP33.21 Traditional Barista Pump Espresso Machine
* DSP espresso machine (KA3113)

These products have received positive reviews from customers, with an average rating of around 3.7 to 4.0 stars out of 5.

Please note that this answer is based solely on the provided data and may not reflect a comprehensive or definitive ranking of coffee makers and espresso machines in Egypt.




🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Could not end session - multiple sessions detected. You must use session.end_session() instead of agentops.end_session() More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
